In [1]:
def getfisher(R, TISSUES):
    O=defaultdict(list)
    Mapdata = {}
    Motifs = list(set(list(R.loc[R['E.tissues']>=1]['motif'])))
    for tis in TISSUES:
        #Review motif policies here
        field= 'eSTR.'+SHORTEN[tis]
        # Focus on estrs and motifs
        data = R[["gene","chrom", "str.start","motif",field]]#,"best.score","best.tissue","INTRON"]]
        data_e = data.loc[data[field]>0]
        #
        #frequent_motifs estrs
        Mo_counts=[list(data['motif']).count(x) for x in Motifs]
        Estr = data_e.shape[0]
        Str = data.shape[0]
        All = []
        # Fisher test for each motif
        for M in Motifs:
            data_eM = data_e.loc[data_e['motif']==M]
            data_M = data.loc[data['motif']==M]
            Mestrs = data_eM.shape[0]
            Mstrs = data_M.shape[0]
            CTT = [[Mestrs, Estr - Mestrs] , [Mstrs-Mestrs , Str-Estr -(Mstrs - Mestrs)]] #contengency
            oddsratio, pv = ss.fisher_exact([CTT[0], CTT[1]])                             #Fisher
            #print ("P (", M, ")    ~~   ",pv*100,"%     with    OR=", oddsratio)
            All.append([M,list(data_e['motif']).count(M),-np.log10(pv),oddsratio])
            O[M].append(oddsratio)
            #
        Mapdata[tis]=All
    #Selecting motifs present in all tissues
    Mset = []
    for o in O.keys():      # O has all the motifs and occurence in tissues
        if len(O[o])>=len(TISSUES):
            Mset.append(o)
    print(len(Mset))
    #heatmap matix
    pdata = np.zeros((len(TISSUES), len(Mset)))
    for i in range(len(TISSUES)):
        T = TISSUES[i]
        for j in range(len(Mset)):
            m=Mset[j]
            val = [s[2] for s in  Mapdata[T] if s[0]==m ]
            if val==[]:
                pdata[i][j] =0
            else:
                pdata[i][j] =val[0]
    return Mset, O,Mapdata, pdata



In [2]:
# Set up
%store -r
%pylab inline
"""
Here i want to look at TF of interest individually

"""
# Libraries
from collections import defaultdict
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats as ss
import seaborn.apionly as sns

#variables
SHORTEN
COLORS
TISSUES
#
#TISSUES = sorted([item for item in list(SHORTEN.keys()) if item not in['LCL','permuted']])
MERGEFILE = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Best_causality.Table'
REGRSSION = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'
OUTDIR = "/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"

Populating the interactive namespace from numpy and matplotlib


In [25]:
%%bash
# Get all STRs analyzed
BASEDIR=/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue
cat ${BASEDIR}/Merged_Best_causality.Table | grep -v gene | awk '{print $2 "\t" $3+0 "\t" $3+1}'|sort | uniq >all_analyzed_strs.bed

In [26]:
mytf = 'MLLT1'                                         #Transcription factor of interest
TFs = '/storage/mgymrek/gtex/tfbs/encodedata/'
transfacts = os.listdir(TFs)
transfacts = [x for x in transfacts if mytf in x][0] 
mytf = TFs+transfacts

In [27]:
#get all overlaps
cmd = "bedtools intersect -a all_analyzed_strs.bed -b %s > Intersect.bed"%MERGEFILE
returned_value = os.system(cmd)
Inter = pd.read_csv('Intersect.bed', sep='\t', header=None, names=["chrom","start","end"])
Feature_strs = list(set(Inter.apply(lambda x: x["chrom"]+":"+str(x["start"]), 1)))

In [6]:
#Look at overlaps and eSTRs
all_strs = pd.read_csv(REGRSSION, sep='\t')
all_strs = all_strs[np.isfinite(all_strs['str.start'])] #Just in case we have NA here. shouldn't
all_strs['str.start']=all_strs['str.start'].astype(int)
all_strs['loci'] = all_strs.apply(lambda x: x["chrom"]+":"+str(x["str.start"]), 1)
data = all_strs.loc[all_strs['loci'].isin(Feature_strs)].copy()
#Causal
merged = pd.read_csv(MERGEFILE, sep="\t")
merged['str.start'] = merged['best.str.start']
merged = merged[np.isfinite(merged['str.start'])]; merged['str.start'] = merged['str.start'].astype(int) 
merged['loci'] = merged["chrom"]+":"+merged["str.start"].astype(str)
mergedFeat = merged.loc[merged['loci'].isin(Feature_strs)].copy()
#check
mergedFeat['causal'] = mergedFeat['top.variant'].apply(lambda x: 1 if "STR" in x else 0)
Causal_strs = list(mergedFeat.loc[mergedFeat['causal']==1]['loci'])                            

In [7]:
#Motif enrichment 
#background = all STR at TF           forground motif enriched at eSTR 
Mset,O,Mapdata,pdata = getfisher(data,TISSUES)
df1 = pd.DataFrame(pdata)
df1.index = [SHORTEN[t] for t in TISSUES]
df1.columns = Mset

0


In [ ]:
#plot enricment (-log10(pvalue) Fisher test)
fig, ax = plt.subplots(figsize=(20, 10))
cmap = sns.diverging_palette(15, 265, s=80, l=55, n=9, as_cmap=True)    
sns.heatmap(df1, cmap=cmap, annot_kws={"size": 8},
            vmin=0.5, vmax=2, square=True, ax=ax, linewidths=0.5, cbar_kws={"shrink": 0.5});
ax.invert_xaxis();
plt.show()

In [ ]:
#plot enricment  log2(odd ratio) of fisher test
odds = np.zeros((len(TISSUES), len(Mset)))
for i in range(len(TISSUES)):
    T = TISSUES[i]
    for j in range(len(Mset)):
        m=Mset[j]
        val = [np.log2(s[3]) for s in  Mapdata[T] if s[0]==m ]
        if val==[]:
            odds[i][j] =0
        else:
            odds[i][j] =val[0]
#
df = pd.DataFrame(odds)
df.index = [SHORTEN[t] for t in TISSUES]
df.columns = Mset
fig, ax = plt.subplots(figsize=(20, 10))
cmap = sns.diverging_palette(15, 265, s=80, l=55, n=9, as_cmap=True)    
sns.heatmap(df, cmap=cmap, annot_kws={"size": 8},
            vmin=0.5, vmax=2, square=True, ax=ax, linewidths=0.5, cbar_kws={"shrink": 0.5});
ax.invert_xaxis();
plt.show()